In [ ]:
# Import libraries
try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load data
train_data = pd.read_csv(train_file_path, sep='\t')
test_data = pd.read_csv(test_file_path, sep='\t')

# Tokenize messages
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['Message'])
train_sequences = tokenizer.texts_to_sequences(train_data['Message'])
test_sequences = tokenizer.texts_to_sequences(test_data['Message'])

# Pad sequences
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post')

# Define and train the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=train_padded.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded, train_data['Category'].apply(lambda x: 1 if x == 'spam' else 0), epochs=5)

# Function to predict messages based on the trained model
def predict_message(pred_text):
    # Tokenize and pad the input message
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_padded = tf.keras.preprocessing.sequence.pad_sequences(pred_sequence, padding='post', maxlen=train_padded.shape[1])

    # Make the prediction
    prediction = model.predict(pred_padded)[0][0]

    # Return the likeliness and label
    if prediction >= 0.5:
        return [prediction, 'spam']
    else:
        return [prediction, 'ham']

# Test the predictions
def test_predictions():
    test_messages = ["how are you doing today",
                    "sale today! to stop texts call 98912460324",
                    "i dont want to go. can we try it a different day? available sat",
                    "our new mobile video service is live. just install on your phone to start watching.",
                    "you have won £1000 cash! call to claim your prize.",
                    "i'll bring it tomorrow. don't forget the milk.",
                    "wow, is your arm alright. that happened to me one time too"
                   ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

# Run the test
test_predictions()
